# Chapter 1

## Fig 1.07 Collins toggle switch

For Figures 1.7, 7.13, 7.14, 7.15

In [ ]:
using DifferentialEquations
using ModelingToolkit
using Plots
Plots.gr(lw=2)

In [ ]:
# Convenience functions
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)

In [ ]:
# Model
@parameters a1 a2 β γ
@variables t s1(t) s2(t) i1(t) i2(t)

D = Differential(t)

# Time-dependent inhibitor levels
i1_fun(t) = ifelse(30<= t <= 40, 10, 0)
i2_fun(t) = ifelse(10<= t <= 20, 10, 0)

@register i1_fun(t)
@register i2_fun(t)

In [ ]:
@named collinsSys = ODESystem([ i1 ~ i1_fun(t),
                                i2 ~ i2_fun(t),
                                D(s1) ~ a1 * hill(1 + i2, s2, β) - s1,
                                D(s2) ~ a2 * hill(1 + i1, s1, γ) - s2])

In [ ]:
collinsSys = structural_simplify(collinsSys)

In [ ]:
u0 = [s1 => 0.075, s2 => 2.5]
p = [a1 => 3.0, a2 => 2.5, β => 4.0, γ => 4.0]
tend = 50.0

In [ ]:
prob = ODEProblem(collinsSys, u0, tend, p)

In [ ]:
sol = solve(prob)

In [ ]:
pl0107a = plot(sol, legend=:right, xlabel = "", ylabel="Concentration", title="Figure 1.7")

# Tracking intermediate variables
pl0107b = plot(sol, vars = [i1, i2], xlabel="Time", ylabel="Strength")

plot(pl0107a, pl0107b, layout=(2, 1))

## Fig 1.09 Hodgkin-Huxley model

In [ ]:
using DifferentialEquations
using ModelingToolkit
using Plots
Plots.gr(lw=2, fmt=:png)

In [ ]:
# Convenience functions
hill(x, k) = x / (x + k)
hill(x, k, n) = hill(x^n, k^n)
exprel(x) = x / expm1(x)

In [ ]:
# Build the model

@parameters E_N E_K E_LEAK G_N_BAR G_K_BAR G_LEAK C_M
@variables t v(t) m(t) h(t) n(t) mα(t) mβ(t) hα(t) hβ(t) nα(t) nβ(t) iNa(t) iK(t) iLeak(t) iStim(t)

D = Differential(t)

In [ ]:
# Time-dependent force
get_istim(t) = ifelse(20<=t<=21, -6.8, 0.0) + ifelse(60<=t<=61, -7.0, 0.0)

@register get_istim(t)

In [ ]:
@named hhSys = ODESystem(
   [mα  ~ exprel(-0.10 * (v + 35)),
    mβ  ~ 4.0 * exp(-(v + 60) / 18.0),
    hα  ~ 0.07 * exp(- ( v + 60) / 20),
    hβ  ~ 1 / (exp(-(v+30)/10) + 1),
    nα  ~ 0.1 * exprel(-0.1 * (v+50)),
    nβ  ~ 0.125 * exp( -(v+60) / 80),
    iNa ~ G_N_BAR * (v - E_N) * (m^3) * h,
    iK  ~ G_K_BAR * (v - E_K) * (n^4),
    iLeak ~ G_LEAK * (v - E_LEAK),
    iStim ~ get_istim(t),
    D(v) ~ -(iNa + iK + iLeak + iStim) / C_M,
    D(m) ~ -(mα + mβ) * m + mα,
    D(h) ~ -(hα + hβ) * h + hα,
    D(n) ~ -(nα + nβ) * n + nα]
)

In [ ]:
hhSys = structural_simplify(hhSys)

In [ ]:
params = [  E_N => 55.0,       # Reversal potential of Na (mV)
            E_K => -72.0,      # Reversal potential of K (mV)
            E_LEAK => -49.0,   # Reversal potential of leaky channels (mV)
            G_N_BAR => 120.0,  # Max. Na channel conductance (mS/cm^2)
            G_K_BAR => 36.0,   # Max. K channel conductance (mS/cm^2)
            G_LEAK => 0.30,    # Max. leak channel conductance (mS/cm^2)
            C_M => 1.0]        # membrane capacitance (uF/cm^2)

u0 = [v => -59.8977, m => 0.0536, h => 0.5925, n => 0.3192]
tend = 100.0

In [ ]:
prob = ODEProblem(hhSys, u0, tend, params, jac=true)

In [ ]:
sol = solve(prob, tstops=[20, 60])

In [ ]:
plot(sol, vars=(t, v),
     ylabel="Membrane potential (mV)", xlabel="Time (ms)", 
     legend=false, title="Figure 1.9")

In [ ]:
plot(sol, vars = [m, h, n], xlabel="Time (ms)")

In [ ]:
plot(sol, vars = [iStim], xlabel="Time (ms)")